> これは[キーボード #1 Advent Calendar 2022](https://adventar.org/calendars/7529)の20日目の記事の一部です。   
> トップページは[Pythonだけでキーボードを作る](https://5z6p.com/2022/12/21/ac2022/)です。

# Make a case

![target case](../imgs/exported_case.png)

Modeling with cadquery to create a case that can be 3D printable.

Since cadquery viewer is not available on google colaboratory due to OS, the notebook is opened with binder.   
The binder enables jupyter notebook to be used by preconfiguring the OS and necessary libraries using Docker.   
Therefore, there is no need to install any libraries.

# What is cadquery?
[cadquery](https://github.com/CadQuery/cadquery) is an environment for coding python to generate 3D models.
Use [jupyter_cadquery](https://github.com/bernhard-42/jupyter-cadquery) as a viewer.

- cadquery https://github.com/CadQuery/cadquery
- cadquery documentation https://cadquery-ja.readthedocs.io/ja/latest/
- jupyter-cadquery https://github.com/bernhard-42/jupyter-cadquery

Run the cell below and make sure the viewer opens.

To run a cell, click on it and press `Shift + Enter` or `▶` on the top menu

In [ ]:
import cadquery as cq

from jupyter_cadquery import (
    versions,
    show, PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer,
    get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay

enable_replay(False)

set_defaults(
    cad_width=640, 
    height=480, 
)

print()
versions()

cv = open_viewer("CadQuery", anchor="right")

## Try cadquery
Generates a simple cube and displays it in the viewer.

In [ ]:
# Make a cube from the xy plane and fillet the edges.
box = cq.Workplane('XY').box(1, 2, 3).edges().fillet(0.1)
# Display in default viewer
show(box)

# Design the case
Design a tray-type case in which PCBs are screwed in place.

## Rough shape
Create a rough outline based on the shape of the PCB. As usual, declare and use constants.

In [ ]:
# PCB Outline
PCB_WIDTH = 76.0
PCB_HEIGHT = 57.0
PCB_THICKNESS = 1.6

# Z margin between case and PCB
CASE_MARGIN_TOP = 11.0
CASE_MARGIN_BOTTOM = 3.5

# XY margin between case and PCB
CASE_MARGIN_PCB = 0.5

# Case Thickness
CASE_FRAME = 2.0
CASE_BOTTOM = 3.0

INNER_HEIGHT = CASE_MARGIN_TOP + PCB_THICKNESS + CASE_MARGIN_BOTTOM
CASE_HEIGHT = INNER_HEIGHT + CASE_BOTTOM

# Draw a rectangle based on the XY plane and extrude it
case = (
    cq.Workplane("XY")
    .rect(
        PCB_WIDTH + (CASE_FRAME + CASE_MARGIN_PCB) * 2,
        PCB_HEIGHT + (CASE_FRAME + CASE_MARGIN_PCB) * 2,
    )
    .extrude(CASE_BOTTOM + INNER_HEIGHT)
    .edges("|Z")
    .fillet(2)
    .edges("|X")
    .chamfer(1)
)

show(case)

## Cut out the inside of the case
Cut out the inside of the case to make a tray shape.   
Begin by selecting the reference plane from the current case face

In [ ]:
# Reference the topmost surface on the Z-axis from the case faces.
# Draw a rectangle and cut off the inside height
case = (
    case.faces(">Z")
    .workplane()
    .rect(PCB_WIDTH + CASE_MARGIN_PCB * 2, PCB_HEIGHT + CASE_MARGIN_PCB * 2)
    .cutBlind(-INNER_HEIGHT)
)

show(case)

## Screw bosses and holes
The reference plane is saved and used with `tag("name")`.   
Prepare an array of screw position coordinates in advance and repeat the same process.

In [ ]:
# Screw coordinates
SCREW_POINTS = [(19, 9.5), (19, -9.5), (-19, -9.5), (0, 9.5)]

# Make the base plane by selecting the top plane on the z-axis (the edge of the case that is just barely left) from the face of the case and offsetting it down by `INNER_HEIGHT`.
# Make a cylinder of radius 2.5mm at each coordinate
# Make a hole of radius 1.1mm at each coordinate
# Make a new reference plane offset 1mm down from the base plane and drill a square hole to the bottom (a hole to put a nut)
case = (
    case.faces(">Z")
    .workplane(offset=-INNER_HEIGHT)
    .tag("InnerBottom")
    .pushPoints(SCREW_POINTS)
    .circle(2.5)
    .extrude(CASE_MARGIN_BOTTOM)
    .workplaneFromTagged("InnerBottom")
    .pushPoints(SCREW_POINTS)
    .circle(1.1)
    .cutThruAll()
    .workplaneFromTagged("InnerBottom")
    .workplane(offset=-1)
    .pushPoints(SCREW_POINTS)
    .rect(4.2, 4.8)
    .cutBlind(-2)
)

show(case)

## USB connector
Make a slot for the USB connector.   
The USB connector needs to be drilled on the rear side, so the reference plane is the furthest plane on the Y-axis.   
There are planes with the same distance in front and behind, however, the plane selected is the rear side, so it is left as it is.

In [ ]:
# USB connector slot location and dimensions
USB_POS = (11.25, 1.2 + 3.15 / 2)
USB_HOLE_SIZE = [9, 3.2, 7.5]
USB_HOLE_MARGIN = 0.5
USB_CONN_SIZE = (11, 8)

# Base on the furthest surface in the Y-axis direction
# Cut out the USB connector slot deep enough to the inside to avoid hitting the USB connector
# Cut out areas to avoid the USB cable housing
case = (
    case.faces(">Y")
    .workplane(centerOption="CenterOfMass")
    .center(
        PCB_WIDTH / 2 - USB_POS[0],
        CASE_HEIGHT / 2 - CASE_MARGIN_TOP - PCB_THICKNESS - USB_POS[1],
    )
    .tag("USBCutout")
    .rect(USB_HOLE_SIZE[0] + USB_HOLE_MARGIN, USB_HOLE_SIZE[1] + USB_HOLE_MARGIN)
    .cutBlind(-(CASE_FRAME + USB_HOLE_SIZE[2] + 2))
    .workplaneFromTagged("USBCutout")
    .rect(USB_CONN_SIZE[0], USB_CONN_SIZE[1])
    .cutBlind(-1)
)

show(case)

## Cover under OLED
Create a cover to be placed between the PCB and the OLED.   
The cover will be made as a separate object, but its position will be aligned.
The cover will be fixed with double-sided tape, so no screws are required.

In [ ]:
OLED_COVER_WIDTH = 19
OLED_COVER_HEIGHT = 19 * 2 - 3
OLED_COVER_THICKNESS = 10

# Base on offset plane from the XY plane by the height above the PCB.
oled_cover = (
    cq.Workplane("XY")
    .workplane(offset=CASE_BOTTOM + CASE_MARGIN_BOTTOM + PCB_THICKNESS)
    .center(-PCB_WIDTH / 2, PCB_HEIGHT / 2)
    .tag("PCB_ORIGIN")
    .center(OLED_COVER_WIDTH / 2, -OLED_COVER_HEIGHT / 2)
    .rect(OLED_COVER_WIDTH, OLED_COVER_HEIGHT)
    .extrude(OLED_COVER_THICKNESS)
    .faces("Z")
    .tag("CASE_TOP")
    .edges(">Y or <X")
    .chamfer(1)
)

show(case, oled_cover)

## Exporting 3D data
Finally, export the 3D model in a data format that can be 3D printed.

In [ ]:
cq.exporters.export(case, "case.stl")
cq.exporters.export(oled_cover, "oled_cover.stl")

# Print the case

Print the stl file generated.

![slicer](../imgs/slicer.png)

![PCB and case](../imgs/case.jpg)

![Connector](../imgs/case_usb.jpg)

After printing, I put the PCB on the case and checked it.
The connector position is perfect.

The case is now complete!